In [2]:
import numpy as np

def EstimateFundamentalMatrix(pointss):
    A = []
    for points in pointss:

        x, y, x_dash, y_dash = points[0], points[1], points[2], points[3]

        A.append([x*x_dash, x*y_dash, x, y*x_dash, y*y_dash, y, x_dash, y_dash, 1])

    A = np.array(A)

    _, __, vt = np.linalg.svd(A)

    v = vt.T
    X = v[:, -1]

    F = np.reshape(X, (3, 3)).T
    U, S, V = np.linalg.svd(F)
    S[2 ]= 0.0
    S_1 = np.diag(S)
    F = U.dot(S_1).dot(V)

    return F

In [3]:
import cv2
import numpy as np
import random

def GetInliersRansac(text_points, threshold = 0.005):
    img1_points = text_points[:, 0:2]
    img2_points = text_points[:, 2:4]

    ones = np.ones((img1_points.shape[0], 1))
    img1_points = np.hstack((img1_points, ones))
    img2_points = np.hstack((img2_points, ones))

    max_inliers = 0

    for i in range(10000):
        points = np.array(random.sample(text_points, 8), np.float32)

        F = EstimateFundamentalMatrix(points)

        values = np.abs(np.diag(np.dot(np.dot(img2_points, F), img1_points.T)))

        index_inliers = np.where(values<threshold)
        index_outliers = np.where(values>=threshold)

        if np.shape(index_inliers[0])[0] > max_inliers:
            max_inliers = np.shape(index_inliers[0])[0]
            index_max_inliers = index_inliers
            index_min_outliers = index_outliers
            F_max_inliers = F

        F_max_inliers = EstimateFundamentalMatrix(text_points[index_max_inliers])

        print("max inliers: {}", (max_inliers))

        return text_points[index_max_inliers], text_points[index_min_outliers], F_max_inliers, text_points[:, 0:2], text_points[:, 2:4]

In [4]:
import numpy as np

def EssentialMatrixFromFundamentalMatrix(F, K):

    E = np.dot(np.dot(K.T, F), K)

    U, S, V_t = np.linalg.svd(E)

    S = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 0]])

    E = np.dot(np.dot(U, S), V_t)

    return E

In [5]:
def CheckDeterminant(C, R):
    if np.linalg.det(R) < 0:
        return -C, -R
    
    else: 
        return C, R

def ExtractCameraPose(E):
    W = np.array([0, -1, 0], [1, 0, 0], [0, 0, 1])

    U, _, V_t = np.linalg.svd(E)

    C_1 = U[:, 2]
    R_1 = (np.dot(U, np.dot(W, V_t)))
    C_2 = -U[:, 2]
    R_2 = (np.dot(U, np.dot(W, V_t)))
    C_3 = U[:, 2]
    R_3 = (np.dot(U, np.dot(W.T, V_t)))
    C_4 = -U[:, 2]
    R_4 = (np.dot(U, np.dot(W.T, V_t)))


    C_1, R_1 = CheckDeterminant(C_1, R_1)
    C_2, R_2 = CheckDeterminant(C_2, R_2)
    C_3, R_3 = CheckDeterminant(C_3, R_3)
    C_4, R_4 = CheckDeterminant(C_4, R_4)


    C = np.array([C_1, C_2, C_3, C_4])
    R = np.array([R_1, R_2, R_3, R_4])

    return C, R

In [6]:
import numpy as np

def LinearTriangulation(M_1, C_2, R_2, K, inliers):

    points_1 = inliers[:, 0:2]
    points_2 = inliers[:, 2:4]

    ones = np.ones((points_1.shape[0], 1))
    points_1 = np.hstack((points_1, ones))
    points_2 = np.hstack((points_2, ones))

    I = np.identity(3)
    M_2 = np.hstack((I, -C_2))
    M_2 = np.dot(K, np.dot(R_2, M_2))

    X_list = []

    for p_1, p_2 in zip(points_1, points_2):
        A = [p_1[0]*M_1[2, :] - M_1[0, :]]
        A.append(p_1[1]*M_1[2, :] - M_1[1, :])
        A.append(p_2[0]*M_2[2, :] - M_2[0, :])
        A.append(p_2[1]*M_2[2, :] - M_2[1, :])

        _, __, V_t = np.linalg.svd(np.array(A))

        V = V_t.T
        X = V[:, -1]

        X = X/X[3]
        X = X[:3]

        X = np.array(X)
        X = np.reshape((3, 1))
        X_list.append(X)
        
    return np.array(X_list)

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
# from LinearTriangulation import linear_triagulation
from Misc.utils import PlotFuncs

def cheirality_check(C, R, X_list):
    count = 0
    r_3 = R[2]

    for X in X_list:
        if(np.dot(r_3, X - C) > 0):
            count+=1
    return count

def DisambiguateCameraPose(M_1, C_2_list, R_2_list, K, inliers):
    count_max = 0
    i = 0
    fg = plt.figure()
    fx = fg.add_subplot(111)
    p_f = PlotFuncs()

    print("Plotting the 4 camera poses ans their points \n")

    for R, C in zip(R_2_list, C_2_list):
        C = C.reshape((3, 1))
        X_list = LinearTriangulation(M_1, C, R, K, inliers)
        count = cheirality_check(C, R, X_list)

        p_f.plot_triangulated_points(X_list, i, C, R)
        print(count)

        if(count > count_max):
            count_max = count
            R_best = R
            C_best = C
            X_list_best = X_list
            index = i

        i+=1
    
    plt.xlim(-15, 20)
    plt.ylim(-30, 40)
    plt.show()

    fig = plt.figure()
    fx = fig.add_subplot(111)
    p_f.plot_triangulated_points(X_list_best, index, C_best, R_best)
    plt.xlim(-15, 20)
    plt.ylim(-30, 40)
    plt.show()

    print("\n")
    return R_best, C_best, X_list_best, index

In [7]:
import numpy as np
from scipy import optimize

def ReprojectionError(points_img1, M_1, points_img2, M_2, X):
    X = np.reshape(X, (3, 1))
    X = np.vstack((X, 1))    

    rep_error = 0

    point_proj_img1 = np.dot(M_1,X)
    point_proj_img1 = point_proj_img1/point_proj_img1[2]

    point_proj_img2 = np.dot(M_2, X)
    point_proj_img2 = point_proj_img2/point_proj_img2[2]

    rep_error += ((points_img1[0] - point_proj_img1[0])**2) + ((points_img1[1] - point_proj_img1)**2)

    rep_error += ((points_img2[0] - point_proj_img2[0])**2) + ((points_img2[1] - point_proj_img2)**2)

    return rep_error

def optimize_params(x0, points_img1, M_1, points_img2, M_2):

    reproj_err = ReprojectionError(points_img1, M_1, points_img2, M_2, x_0)
    return reproj_err


def NonlinearTriangulation(M_1, M_2, X_list, inliers, K):
    points_img1 = inliers[:, 0:2]
    points_img2 = inliers[:, 2:4]

    X_list_ref = []

    for point_img1, point_img2 in  zip(points_img1, points_img2):
        X = X.reshape(X.shape[0],)
        res = optimize.least_squares(fun=optimize_params, x0=X, args = [point_img1, M_1, point_img2, M_2])
        x_ref = res.x
        x_ref = np.reshape(X, (3,))
        X_list_ref.append(x_ref)
    X_list_ref = np.array(X_list_ref)
    X_list_ref = X_list_ref.reshape((X_list_ref.shape[0], 3))
    

In [9]:
import numpy as np

def LinearPnP(img_2d_3d, K):
    A = np.empty((0, 12), np.float32)
    for img_pt in img_2d_3d:
        x, y = img_pt[0], img_pt[1]
        normalised_pts = np.dot(np.linalg.inv(K), np.array([[x], [y], [1]]))
        normalised_pts = normalised_pts/normalised_pts[2]

        X = img_pt[2:]
        X = X.reshape((3, 1))

        X = np.append(X, 1)

        Z = np.zeros((4, ))
        A_1 = np.hstack((Z, -X.T, normalised_pts[1]*(X.T)))
        A_2 = np.hstack((X.T, Z, -normalised_pts[0]*(X.T)))
        A_3 = np.hstack((-normalised_pts[1]*(X.T), normalised_pts[0]*X.T, Z))

        for a in [A_1, A_2, A_3]:
            A = np.append(A, [a], axis = 0)

    A = np.float32(A)
    _, __, V_t = np.linalg.svd(A)

    V = V_t.T

    pose = (V[:, -1]).reshape((3, 4))

    R_ = (pose[:, 0:3]).reshape((3, 3))
    T_ = (pose[:, 3]).reshape((3, 1))

    U, __, V_t = np.linalg.svd(R_)
    R_ = np.dot(U, V_t)

    if np.linalg.det(R_) < 0 :
        R_ = -R_
        T_ = -T_

    C_ = -np.dot(R_, T_)
    return R_, C_

    

In [11]:
import numpy as np
import random

def AllReprojetionError(img_points, M, X_all, ret = False):
    o = np.ones((X_all.shape[0], 1))
    X_all = (np.hstack((X_all, o))).T
    
    proj_points = (np.dot(M, X_all)).T
    proj_points[:, 0] = proj_points[:, 0]/proj_points[:, 2]
    proj_points[:, 1] = proj_points[:, 1]/proj_points[:, 2]
    proj_points = proj_points[:, 0:2]

    reproj_error = (img_points - proj_points)**2
    reproj_error = np.sum(reproj_error, axis=1)

    if(ret):
        return reproj_error, proj_points
    return reproj_error


def PnPRANSAC(correspond_2d_3d, K, threshold = 20):
    max_inliers = 0

    for i in range(10000):
        corr6 = np.array(random.sample(correspond_2d_3d, 6), np.float32)
        R, C = LinearPnP(corr6, K)
        C = C.reshape((3, 1))
        I = np.identity(3)
        M = np.hstack((I, -C))
        M = np.dot(K, np.dot(R, M))

        img_points = correspond_2d_3d[:, 0:2]
        X_all = correspond_2d_3d[:, 2]

        reproj_error = AllReprojetionError(img_points, M, X_all)
        loc = np.where(reproj_error < threshold)[0]
        count = np.shape(loc)[0]
        if count > max_inliers:
            max_inliers = count
            inliers = correspond_2d_3d[loc]
            R_ = R
            C_ = C

    best_pose = np.hstack((R_, C_))
    print("Max inliers: ")
    print(max_inliers)

    return best_pose, inliers

In [1]:
from Misc.utils import MiscFuncs
import numpy as np
from scipy.optimize import least_squares

def ComputeProjError(M, image_point, X):

    X = X.reshape((3, 1))
    X = np.append(X, 1)

    project_impage_point = np.dot(M, X)
    project_impage_point[0] = project_impage_point[0]/project_impage_point[2]
    project_impage_point[1] = project_impage_point[1]/project_impage_point[2]

    reproj_error = ((image_point[0] - project_impage_point[0])**2) + ((image_point[1] - project_impage_point[1])**2)

def rot_to_quat(R):
	qxx,qyx,qzx,qxy,qyy,qzy,qxz,qyz,qzz = R.flatten()
	m = np.array([[qxx-qyy-qzz,0, 0, 0],[qyx+qxy,qyy-qxx-qzz,0,0],
		[qzx+qxz,qzy+qyz,qzz-qxx-qyy,0],[qyz-qzy,qzx-qxz,qxy-qyx,qxx+qyy+qzz]])/3.0
	val,vec = np.linalg.eigh(m)
	quat = vec[[3,0,1,2],np.argmax(val)]
	if quat[0]<0:
		quat = -quat

	return quat 

def quat_to_rot(quat):
	q_1,q_2,q_3,q_4 = quat
	Nq = q_1*q_1+q_2*q_2+q_3*q_3+q_4*q_4
	if Nq < np.finfo(np.float).eps:
		return np.eye(3)
	s = 2.0/Nq
	X = q_2*s
	Y = q_3*s
	Z = q_4*s
	wX = q_1*X; wY = q_1*Y; wZ = q_1*Z
	xX = q_2*X; xY = q_2*Y; xZ = q_2*Z
	yY = q_3*Y; yZ = q_3*Z; zZ = q_4*Z
	R =  np.array([[ 1.0-(yY+zZ), xY-wZ, xZ+wY ],
            [ xY+wZ, 1.0-(xX+zZ), yZ-wX ],
            [ xZ-wY, yZ+wX, 1.0-(xX+yY) ]])

	return R	

def NonlinearPnP(K, pose, corresp_2d_3d):
	poses_non_lin = {}
	corr = corresp_2d_3d
	img_points = corr[:, 0:2]
	X_all = corr[:, 2:]
	R = pose[:, 0:3]
	C = (pose[:, 3]).reshape((3, 1))
	Quat = rot_to_quat(R)
	x0 = np.append(Quat, C)
	res = least_squares(fun = optimize_params, x0 = x0, args = (K, img_points, X_all), ftol = 1e-10)
	O = res.x
	R_best = quat_to_rot(O[:4])
	C_best = (O[4:]).reshape((3, 1))
	pose_best = np.hstack((R_best, C_best))
	return pose_best
	
    

SyntaxError: incomplete input (2837690497.py, line 1)

In [ ]:
from scipy.sparse import lil_matrix

def BuildVisibilityMatrix(n_camreas, n_points, cam_indices, point_indices):
    r = cam_indices.size * 2
    c = n_camreas * 7 + n_points * 3
    V = lil_matrix((r, c), dtype=int)

    i = np.arange(cam_indices.size)

    for s in range (7):
        V[2 * i, cam_indices * 7 + s] = 1
        V[2 * i + 1, cam_indices * 7 + s] = 1

    for s in range (3):
        V[2 * i, cam_indices * 3 + s] = 1
        V[2 * i + 1, cam_indices * 3 + s] = 1

    return V    

In [ ]:
import numpy as np
from scipy.optimize import least_squares
from NonlinearPnP import rot2Quat, quat2Rot
from Misc.utils import MiscFuncs
from scipy.sparse import lil_matrix
from BuildVisibilityMatrix import bundle_adjustment_sparsity
import time


def reproj_error(K, img_pts_2d, param_cam, world_pts_3d):

    misc_funcs = MiscFuncs()
    ones = np.ones((world_pts_3d.shape[0], 1))
    world_pts_3d = np.hstack((world_pts_3d, ones))

    pt_img_proj = np.empty((0, 2), dtype=np.float32)

    for i, p in enumerate(world_pts_3d):
        R = quat2Rot(param_cam[i, :4])
        C = param_cam[i, 4:]
        M = misc_funcs.get_projection_matrix(K, R, C)
        p = p.reshape((4, 1))
        proj_pt = np.dot(M, p)
        proj_pt = proj_pt/proj_pt[2]
        proj_pt = proj_pt[:2]
        proj_pt = proj_pt.reshape((1, 2))
        pt_img_proj = np.append(pt_img_proj, proj_pt, axis=0)

    reproj_err = img_pts_2d - pt_img_proj
    return reproj_err.ravel()


def bundle_adj_params(pose_set, X_world_all, map_2d_3d):

    # defining the params, 2d points, 3d points' indices
    x0 = np.empty(0, dtype=np.float32)
    indices_3d_pts = np.empty(0, dtype=int)
    img_pts_2d = np.empty((0, 2), dtype=np.float32)
    indices_cam = np.empty(0, dtype=int)

    n_cam = max(pose_set.keys())

    # for each camera pose
    for k in pose_set.keys():

        # convert to quaternion
        Q = rot2Quat(pose_set[k][:, 0:3])
        C = pose_set[k][:, 3]
        # append the parameters
        x0 = np.append(x0, Q.reshape(-1), axis=0)
        x0 = np.append(x0, C, axis=0)

        for p in map_2d_3d[k]:
            indices_3d_pts = np.append(indices_3d_pts, [p[1]], axis=0)
            img_pts_2d = np.append(img_pts_2d, [p[0]], axis=0)
            indices_cam = np.append(indices_cam, [k-1], axis=0)

    x0 = np.append(x0, X_world_all.flatten(), axis=0)
    n_3d = X_world_all.shape[0]

    return n_cam, n_3d, indices_3d_pts, img_pts_2d, indices_cam, x0


def optimize(x0, n_cam, n_3d, indices_3d_pts, img_pts_2d, indices_cam, K):

    param_cam = x0[:n_cam*7].reshape((n_cam, 7))
    world_pts_3d = x0[n_cam*7:].reshape((n_3d, 3))
    reproj_err = reproj_error(K, img_pts_2d, param_cam[indices_cam], world_pts_3d[indices_3d_pts])
    return reproj_err


def bundle_adjustment(pose_set, X_world_all, map_2d_3d, K):


    n_cam, n_3d, indices_3d_pts, img_pts_2d, indices_cam, x0 = bundle_adj_params(pose_set, X_world_all, map_2d_3d)

    A = bundle_adjustment_sparsity(n_cam, n_3d, indices_cam, indices_3d_pts)

    start = time.time()
    result = least_squares(fun=optimize, x0=x0, jac_sparsity=A, verbose=2, x_scale='jac',
    ftol=1e-4, method='trf', args=(n_cam, n_3d, indices_3d_pts, img_pts_2d, indices_cam, K))
    end = time.time()
    print("optimisation took -- {} seconds".format(start-end))

    param_cam = result.x[:n_cam*7].reshape((n_cam, 7))
    X_world_all_opt = result.x[n_cam*7:].reshape((n_3d, 3))
    pose_set_opt = {}
    i = 1
    for cp in param_cam:
        R = quat2Rot(cp[:4])
        C = cp[4:].reshape((3, 1))
        pose_set_opt[i] = np.hstack((R, C))
        i += 1

    return pose_set_opt, X_world_all_opt

In [ ]:
def main():